# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
## <center> **Final Project (Website Activity) Consumer** </center>


**Team**:
- Luis Raul Acosta Mendoza 
- Samantha Abigail Quintero 
- Arturo Benajamin Vergara Romo
    
**Profesor**: Dr. Pablo Camarillo Ramirez

## Overview
This notebook consumes streaming data from Kafka topics and processes it using Spark Structured Streaming.


## 1. Environment Setup

In [13]:
import findspark
findspark.init()

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from gatubelxs.spark_utils import SparkUtils
from gatubelxs.streaming_monitor import StreamingPerformanceMonitor, analyze_performance_data

## 2. Directory Structure

In [24]:
import os 
base_path = "/home/jovyan/notebooks/data"

for topic in ["page_views", "click_events", "user_interactions"]:
    os.makedirs(f"{base_path}/{topic}/checkpoint", exist_ok=True)
    os.makedirs(f"{base_path}/{topic}", exist_ok=True)  # For the data files too

## 3. Schema Definitions

In [16]:
spark = SparkSession.builder \
    .appName("FinalProjectGatubelxs") \
    .master("spark://b33dcc1265b4:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

In [17]:
performance_monitor = StreamingPerformanceMonitor()
spark.streams.addListener(performance_monitor)

In [18]:
TOPICS = ["page_views", "click_events", "user_interactions"]
KAFKA_BOOTSTRAP_SERVER = "kafka:9093"

In [19]:
page_views_schema = SparkUtils.generate_schema(
    [

        ("user_id", "string"),
        ("session_id", "string"),
        ("page_url", "string"),
        ("referrer_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("timestamp", "string")
    ]
)

In [20]:
click_events_schema = SparkUtils.generate_schema(
    [
        ("user_id", "string"),
        ("session_id", "string"),
        ("element_id", "string"),
        ("page_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("timestamp", "string"),
        ("x_coord", "float"),
        ("y_coord", "float"),
    ]
)

In [21]:
user_interaction_schema = SparkUtils.generate_schema(
    [
        ("user_id", "string"),
        ("session_id", "string"),
        ("interaction_type", "string"),
        ("page_url", "string"),
        ("category", "string"),
        ("price", "float"),
        ("details", "string"),
        ("timestamp", "string")
    ]
)

## 4. Stream Processing Setup

In [ ]:
def create_stream_query(topic, schema, output_dir):
    kafka__stream = spark.readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", KAFKA_BOOTSTRAP_SERVER) \
        .option("subscribe", topic) \
        .load()
    
    parsed_data = kafka__stream.selectExpr("CAST(value as STRING) as json") \
        .withColumn("data", from_json(col("json"), schema)) \
        .select("data.*")

    return parsed_data.writeStream \
        .format("parquet") \
        .outputMode("append") \
        .option("path", f"{base_path}/{topic}") \
        .option("checkpointLocation", f"{base_path}/{topic}/checkpoint") \
        .trigger(processingTime="5 seconds") \
        .start()



Performance summary for page_views]:
Processed 5 rows at 5.02 rows/second
Input rate: 1.00 rows/second
Processing time: 997ms

Performance summary for page_views]:
Processed 5 rows at 5.02 rows/second
Input rate: 1.00 rows/second
Processing time: 997ms

Performance summary for user_interactions]:
Processed 2 rows at 1.24 rows/second
Input rate: 0.40 rows/second
Processing time: 1615ms

Performance summary for user_interactions]:
Processed 2 rows at 1.24 rows/second
Input rate: 0.40 rows/second
Processing time: 1615ms

Performance summary for click_events]:
Processed 7 rows at 4.07 rows/second
Input rate: 1.40 rows/second
Processing time: 1721ms

Performance summary for click_events]:
Processed 7 rows at 4.07 rows/second
Input rate: 1.40 rows/second
Processing time: 1721ms


## 5. Start Streaming Queries and performance monitoring

In [23]:
queries = [
    create_stream_query("page_views", page_views_schema, "./datalake/page_views"),
    create_stream_query("click_events", click_events_schema, "./datalake/click_events"),
    create_stream_query("user_interactions", user_interaction_schema, "./datalake/user_interactions"),
]

for q in queries:
    q.awaitTermination(timeout=60)

for q in queries:
    q.stop()

analyze_performance_data(performance_monitor.metrics_history)

25/05/13 03:51:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/13 03:51:44 WARN StreamingQueryManager: Stopping existing streaming query [id=e911c040-17ae-4a26-91c1-e4120ae91dd2, runId=50c565fe-d9ef-4b68-8a86-6d5f2affae34], as a new run is being started.
25/05/13 03:51:44 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/13 03:51:44 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/13 03:51:44 WARN StreamingQueryManager: Stopping existing streaming query [id=1fcf2acb-4a9a-4f03-bc11-6f56fdac6d05, runId=5a9700ed-94f0-4836-aef6-1b7619d5a4a1], as a new run is being started.
25/05/13 03:51:44 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.comm

Query terminated: e911c040-17ae-4a26-91c1-e4120ae91dd2 at 2025-05-13 03:51:44.714134
Query started: e911c040-17ae-4a26-91c1-e4120ae91dd2 at 2025-05-13 03:51:44.714594
Query terminated: e911c040-17ae-4a26-91c1-e4120ae91dd2 at 2025-05-13 03:51:44.715248
Query started: e911c040-17ae-4a26-91c1-e4120ae91dd2 at 2025-05-13 03:51:44.715379
Query terminated: 1fcf2acb-4a9a-4f03-bc11-6f56fdac6d05 at 2025-05-13 03:51:44.765407
Query started: 1fcf2acb-4a9a-4f03-bc11-6f56fdac6d05 at 2025-05-13 03:51:44.765779
Query terminated: 1fcf2acb-4a9a-4f03-bc11-6f56fdac6d05 at 2025-05-13 03:51:44.766653
Query started: 1fcf2acb-4a9a-4f03-bc11-6f56fdac6d05 at 2025-05-13 03:51:44.766721


25/05/13 03:51:44 WARN StreamingQueryManager: Stopping existing streaming query [id=3112a589-5021-474c-8466-77a6354c0d53, runId=8cceec6d-4289-4e5b-9e61-692b5a07ce9f], as a new run is being started.
25/05/13 03:51:44 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


Query terminated: 3112a589-5021-474c-8466-77a6354c0d53 at 2025-05-13 03:51:44.934728
Query started: 3112a589-5021-474c-8466-77a6354c0d53 at 2025-05-13 03:51:44.936048
Query started: 3112a589-5021-474c-8466-77a6354c0d53 at 2025-05-13 03:51:44.941141
Query terminated: 3112a589-5021-474c-8466-77a6354c0d53 at 2025-05-13 03:51:44.943564



Performance summary for page_views]:
Processed 3 rows at 2.96 rows/second
Input rate: 0.00 rows/second
Processing time: 1014ms

Performance summary for page_views]:
Processed 3 rows at 2.96 rows/second
Input rate: 0.00 rows/second
Processing time: 1014ms


...


Performance summary for user_interactions]:
Processed 6 rows at 6.64 rows/second
Input rate: 1.20 rows/second
Processing time: 904ms

Performance summary for user_interactions]:
Processed 6 rows at 6.64 rows/second
Input rate: 1.20 rows/second
Processing time: 904ms

Performance summary for page_views]:
Processed 13 rows at 9.00 rows/second
Input rate: 2.60 rows/second
Processing time: 1445ms

Performance summary for page_views]:
Processed 13 rows at 9.00 rows/second
Input rate: 2.60 rows/second
Processing time: 1445ms

Performance summary for click_events]:
Processed 16 rows at 22.92 rows/second
Input rate: 3.20 rows/second
Processing time: 698ms

Performance summary for click_events]:
Processed 16 rows at 22.92 rows/second
Input rate: 3.20 rows/second
Processing time: 698ms

Performance summary for user_interactions]:
Processed 25 rows at 19.31 rows/second
Input rate: 5.00 rows/second
Processing time: 1295ms

Performance summary for user_interactions]:
Processed 25 rows at 19.31 row


Performance summary for click_events]:
Processed 19 rows at 22.09 rows/second
Input rate: 3.80 rows/second
Processing time: 860ms

Performance summary for click_events]:
Processed 19 rows at 22.09 rows/second
Input rate: 3.80 rows/second
Processing time: 860ms

Performance summary for page_views]:
Processed 17 rows at 11.83 rows/second
Input rate: 3.40 rows/second
Processing time: 1437ms

Performance summary for page_views]:
Processed 17 rows at 11.83 rows/second
Input rate: 3.40 rows/second
Processing time: 1437ms

Performance summary for user_interactions]:
Processed 17 rows at 8.56 rows/second
Input rate: 3.40 rows/second
Processing time: 1985ms

Performance summary for user_interactions]:
Processed 17 rows at 8.56 rows/second
Input rate: 3.40 rows/second
Processing time: 1985ms

Performance summary for page_views]:
Processed 14 rows at 34.91 rows/second
Input rate: 2.80 rows/second
Processing time: 401ms

Performance summary for page_views]:
Processed 14 rows at 34.91 rows/second



Performance summary for click_events]:
Processed 15 rows at 20.41 rows/second
Input rate: 3.00 rows/second
Processing time: 735ms

Performance summary for click_events]:
Processed 15 rows at 20.41 rows/second
Input rate: 3.00 rows/second
Processing time: 735ms

Performance summary for user_interactions]:
Processed 16 rows at 12.02 rows/second
Input rate: 3.20 rows/second
Processing time: 1331ms

Performance summary for user_interactions]:
Processed 16 rows at 12.02 rows/second
Input rate: 3.20 rows/second
Processing time: 1331ms

Performance summary for page_views]:
Processed 17 rows at 12.10 rows/second
Input rate: 3.40 rows/second
Processing time: 1405ms

Performance summary for page_views]:
Processed 17 rows at 12.10 rows/second
Input rate: 3.40 rows/second
Processing time: 1405ms

Performance summary for user_interactions]:
Processed 9 rows at 18.83 rows/second
Input rate: 1.80 rows/second
Processing time: 478ms

Performance summary for user_interactions]:
Processed 9 rows at 18.8


Performance summary for page_views]:
Processed 15 rows at 9.20 rows/second
Input rate: 3.00 rows/second
Processing time: 1630ms

Performance summary for page_views]:
Processed 15 rows at 9.20 rows/second
Input rate: 3.00 rows/second
Processing time: 1630ms
Query terminated: e911c040-17ae-4a26-91c1-e4120ae91dd2 at 2025-05-13 03:54:44.975262
Query terminated: e911c040-17ae-4a26-91c1-e4120ae91dd2 at 2025-05-13 03:54:44.978371
Query terminated: 1fcf2acb-4a9a-4f03-bc11-6f56fdac6d05 at 2025-05-13 03:54:44.980414

Performance Analysis fro unknown:

Average processing rate: 11.89 rows/second
Peak processing rate: 54.71 rows/second
Average processing time: 1230.55ms
Peak processing time: 2086ms
Query terminated: 1fcf2acb-4a9a-4f03-bc11-6f56fdac6d05 at 2025-05-13 03:54:44.989903


Query terminated: 3112a589-5021-474c-8466-77a6354c0d53 at 2025-05-13 03:54:44.996579
Query terminated: 3112a589-5021-474c-8466-77a6354c0d53 at 2025-05-13 03:54:45.000718


In [ ]:
sc.stop()